In [1]:
import numpy as np
import pylab as plt
import casatools as tools
import os
from casa_utils import applycal, makeimage, fitimage, getimage, findfrb, image_summary

In [ ]:
msfile1 = 'file.ms'

In [ ]:
calpath = '/path/to/cal/products/'
prename  = 'name.ms.'

In [ ]:
msm = tools.msmetadata()
msm.open(msfile1)
fieldnames = msm.fieldnames()
print(fieldnames)

In [ ]:
targetfield = str(fieldnames.index(fieldnames[0]))

In [ ]:
caltables = ['hifv_priorcals.s5_2.gc.tbl',
             'hifv_priorcals.s5_3.opac.tbl',
             'hifv_priorcals.s5_4.rq.tbl',
             'hifv_finalcals.s14_2.finaldelay.tbl',
             'hifv_finalcals.s14_4.finalBPcal.tbl',
             'hifv_finalcals.s14_5.averagephasegain.tbl',
             'hifv_finalcals.s14_7.finalampgaincal.tbl',
             'hifv_finalcals.s14_8.finalphasegaincal.tbl']

In [ ]:
gaintables = [calpath + '/' + prename + caltable for caltable in caltables]

## Calibrate fast SDM

In [ ]:
applycal(msfile1, gaintables, targetfield=targetfield,
         gainfield=['', '', '', '', '', '', '', ''],
         interp=['linear', 'linear', 'linear', 'linear', 'linear,linearflag', 'linear', 'linear', 'linear'])

## Image fast SDM (CASA)

In [ ]:
if msfile1[-1] == '/':
    file = msfile1[:-1]
else:
    file = msfile1
basename, ext = os.path.splitext(file)
image_name = 'images_FRB_' + '_'.join(basename.split('_')[-2:])

In [ ]:
name = image_name

In [ ]:
fieldnames[0]

In [ ]:
snr = 0
for i in range(2, 12):
    for j in range(2, 12):
        if j > i:
            spws = str(i) + '~' + str(j)
            snr_ret = findfrb(name, msfile1, fieldnames[0], spws, npix=2048)
            if snr_ret > snr:
                snr = snr_ret
                sp = spws

In [ ]:
print(snr, sp)

In [ ]:
!rm -rf {name}*
makeimage(msfile1, fieldnames[0], outname=name,  spw=sp, niter=100, cell=0.25, npix=4096)

#### Make sure the rough peak position reported here is similar to the candidate position reported by rfpipe. If not, there might be a bright blob leading to this peak detection. Plot the image, select a part of it which contains the candidate (and not this blob) and use that in fitimage and findfrb. This can happen for weak candidates.

In [ ]:
imvals, _ , _ = image_summary(f'{name}.image')

In [ ]:
imvals = getimage(f'{name}.image')
npixx,npixy = imvals.shape

In [ ]:
peakx, peaky = np.where(imvals.max() == imvals)
peakx, peaky = peakx[0], peaky[0]
print(peakx, peaky)

In [ ]:
size =  800
fig = plt.figure(figsize=(15,8))
ax = fig.add_axes()
plt.imshow(imvals[peakx-size//2:peakx+size//2, 
                 peaky-size//2:peaky+size//2].transpose(),
          interpolation='nearest', origin='bottom')
plt.colorbar()
plt.xlabel("RA (pixels)")
plt.ylabel("Dec (pixels)")

In [ ]:
az, el, az_err, el_err = fitimage(f'{name}.image', outname=f'{name}.', fitwindow=50)